In [1]:
import os
import re
import json
import random
import sys
import pickle
import datetime
import asyncio
import nest_asyncio
import sympy as sp
# import gurobipy as gp

from openai import OpenAI, AsyncClient
from json import JSONDecodeError
from tqdm.auto import tqdm
from colorama import Fore, Style
from pydantic import BaseModel
from typing import List
from llama_index.core.program import LLMTextCompletionProgram
from llama_index.llms.lmstudio import LMStudio

sys.path.append('../')
from utils import *

nest_asyncio.apply()

c:\Users\s381731\Miniconda3\envs\age\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\s381731\Miniconda3\envs\age\Lib\site-packages\pydantic\_internal\_fields.py:161: UserWarning: Field "model_name" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [2]:
DATA_DIR = '../data'
DATASET_NAME = 'LPWP' 
OUTPUT_DIR = '../output'  

dt = datetime.datetime.today().strftime('%Y-%m-%d-%H-%M-%S')

nl4opt_data = read_txt_file(os.path.join(DATA_DIR, DATASET_NAME, 'LPWP.txt'))
questions, answers = get_nl4opt_qas(nl4opt_data)
assert len(questions) == len(answers)

qa_pairs = list(zip(questions, answers))
# demo_samples, test_samples = get_demo_and_test_samples(qa_pairs)

questions = [q for q, _ in qa_pairs]
answers = [a for _, a in qa_pairs]

2024-09-20 19:35:08.130 | DEBUG    | utils:read_txt_file:15 - Reading file: ../data\LPWP\LPWP.txt
2024-09-20 19:35:08.132 | DEBUG    | utils:read_txt_file:17 - File read successfully: ../data\LPWP\LPWP.txt
2024-09-20 19:35:08.135 | INFO     | utils:get_nl4opt_qas:36 - Number of questions: 288
2024-09-20 19:35:08.136 | INFO     | utils:get_nl4opt_qas:37 - Number of answers: 288


In [41]:
llm = LMStudio(
    model_name="lmstudio-community/Meta-Llama-3.1-8B-Instruct-GGUF",
    base_url="http://localhost:1234/v1",
    temperature=0.0,
)

In [42]:
client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

In [43]:
prompt_template_str = """You are an expert in optimization problems. Your task is to find the optimal solution to the given problem. If there is no optimial solution, please return inf. Guide the user through the solution step by step.\n"""
print(prompt_template_str)

You are an expert in optimization problems. Your task is to find the optimal solution to the given problem. If there is no optimial solution, please return inf. Guide the user through the solution step by step.



In [44]:
lp_result_list = []
for i in tqdm(range(len(questions))):
  response = client.chat.completions.create(
        model="lmstudio-community/DeepSeek-Coder-V2-Lite-Instruct-GGUF",
        messages=[ 
          { "role": "system", "content": prompt_template_str },
          { "role": "user", "content": f"QUESTION: {questions[i]}" }
        ], 
        response_format={
            "type": "json_schema",
            "json_schema": {
            "strict": "true",
            "schema": {
              "type": "object",
              "properties": {
                "rea": {
                  "type": "string",
                  "description": "The reasoning process to find the optimal solution."
                },
                "res": {
                  "type": "number",
                  "description": "The final optimal answer to the question."
                }
              },
            "required": ["res"]
            }
          }
        },
        temperature=0.7, 
        max_tokens=-1,
        stream=False,
    )
  
  lp_result_list.append(response)

100%|██████████| 21/21 [01:45<00:00,  5.00s/it]


In [46]:
lp_results = []
for lp_r in lp_result_list:
    lp_json = json.loads(lp_r.choices[0].message.content)
    lp_results.append(lp_json['res'])

In [47]:
filename = 'deepseek_coderv2_lpwp_gurobi_baseline_' + dt + '.pkl'
with open(os.path.join(OUTPUT_DIR, filename), 'wb') as f:
    pickle.dump(lp_results, f)

In [3]:
filename = 'deepseek_coderv2_lpwp_gurobi_baseline_2024-09-20-19-25-11.pkl'
with open(os.path.join(OUTPUT_DIR, filename), 'rb') as f:
    pred_answers = pickle.load(f)

In [4]:
answers = [a if a != 'None' else str(np.inf) for a in answers]

In [5]:
assert len(pred_answers) == len(answers)

In [6]:
print(pred_answers)

[1, 6100, 235, 0, 0, 10, 1, 0, 100, 50, 2, 20, 1, 1, 0, 1, 1, 20000, 1, 1, 0, 120, 268, 1.0, 1, 0, 1, 1.0, 1, 1, 100, 1, 588, 1, 1, 1, 0, 0, 1, 1, 1.0, 1, 0, 1, 0, 1, 0, 0, 0, 3, 1, 0, 0, 0, 1, 568, 1.25, 2, 2500, 1.29, 12, 123456789.0, -1, 1, 1, 1.25, 4, 3, 1.0, 0, 1, 1.125, 2, 4, 524, 1, 4, 1, 750, 522.02, 1, 2, 1.2345, 1, 12, 1, 3, 2, 0, 1, 340, 0, 1000, 1, 1, 1.0, 0, 1, 1, 168, 0, 1, 1.5, 1, 1, 0, 1, 1.0, 1, 250, 1, 30, 0, 0, 666.67, 1, 11750, 1, 0, 1, 890, 350, 1, 15, 11000, 0, 1, 1, 1, 1, 0, 1, 7.95, 1, 15, 1, 1, 1, 1, 1, 1750, 1, 1, -1, 5, 0, 14, 0, 1, 1, 0, 1, 1.02, 1, 4, 6400, 1, 6, 1.05, 1, 17, 0, 1, 1, 1, 10000, 1, 4, 1, 1, 1, 1, 123456, 1, 1, 1, 1, 1500, 0, 1, 25, 1, 12, 1, 1.125, 1, 1, 1, 1, 1, 45, 1, 0, 1, 1.1, 1, 0.9, 1, 180, 1.0, 0, 1, 4, 1570800.0, 1.5, 1, 0, 1, 1, 0, -1, 160, 1, 1, 91000, 1, 7.5, 1, 1, 650, 1, 4400, 1000, 12, 160000, 10, 1, 600, 1, 30, 1, 1, 159000, -1, 2, 1.0, 5, 1, 24, 1760, 2, 103527, 1.75, 1, 0, 0, 0, 0, 1, 1, 1200, 1, 150, 1, 1, 150, 1, 1, 1, 1, 

In [7]:
print(answers)

['1300', '5050', '125.4929565', '3', '110.0', '45.0', '810.0', '582.0', '1400.0', '36.0', '96', '0', '0.0', '684000.0', '4.0', '64', '125.0', '2500', '98.0', '62.5', '0.0', '160', '226.0', '5625', '60', '1.5', '750.0', '200', '735.0', '360', '368', '0', '603.0', '1000.0', '4347.82612', '899975', '20.8', '440.0', '60', '1266', '2000', '87.5', '1500.0', '105', '68.0', 'inf', '1140', '1.5', '72', '50.0', '350', '833.3333299999999', '0.0', '390', '40', '571.0', '24.0', '0.0', '1990', '6300.0', '1160.0', '2200', '2480', '71.0', '14.0', '60', '20.0', '75.0', '4000000.0', '465.0', '580.0', '555', '120.0', '1970.0', '513', '16.0', '67.0', '75', '19.0', '224.0', '4190.0', '600.0', '33', '150', '540', '342.0', 'inf', '19.0', '80', '26.0', '327.65957199999997', '1480.0', '310.0', '430.0', '175.0', '960', '2500', '29.0', '32', '78', '45', '239.0', '16500', '230.0', '24', '670', '420.0', '841.0', 'inf', '89.0', '37.0', '23', '150000.0', '14', '342750', '133200', '11980', '30000.0', '3.0', '20.0', '

In [8]:
def mark(pred, real, error: float) -> List[bool]:    
    correct = []
    for p, r in zip(pred, real):
        if p == 'Error':
            continue
        if float(r) != 0:
            if (float(p) == np.inf and float(r) == np.inf) or (abs(float(p) - float(r)) / float(r) < error):
                correct.append(True)
            else:
                correct.append(False)
        else:
            if float(p) < error:
                correct.append(True)
            else:
                correct.append(False)
    return correct

In [9]:
print(f"Accuracy under error {1e-2}: {sum(mark(pred_answers, answers, 1e-2)) / len(answers) * 100}")
print(f"Accuracy under error {1e-4}: {sum(mark(pred_answers, answers, 1e-4)) / len(answers) * 100}")
print(f"Accuracy under error {1e-6}: {sum(mark(pred_answers, answers, 1e-6)) / len(answers) * 100}")

Accuracy under error 0.01: 5.555555555555555
Accuracy under error 0.0001: 5.208333333333334
Accuracy under error 1e-06: 5.208333333333334
